In [1]:
import os
import platform 
import tensorflow as tf 
import matplotlib.pyplot as plt 
import numpy as np 

In [2]:
mnist = tf.keras.datasets.mnist
print("loading")
(x_train, y_train),(x_test, y_test) = mnist.load_data()
print("loaded")
x_train, x_test = x_train/255.0, x_test/255.0 
x_train=np.expand_dims(x_train, axis=3) 
x_test=np.expand_dims (x_test, axis=3)
print("preparing model")

loading
loaded
preparing model


In [3]:
y_train[0]

5

In [4]:
plt.ioff()
mnist = tf.keras.datasets.mnist
print("loading")
(x_train, y_train),(x_test, y_test) = mnist.load_data()
print("loaded")
x_train, x_test = x_train/255.0, x_test/255.0 
x_train=np.expand_dims(x_train, axis=3) 
x_test=np.expand_dims (x_test, axis=3)
print("preparing model")
model = tf.keras.models.Sequential([ 
tf.keras.layers.Conv2D(32, kernel_size=(3,3), input_shape=x_train[0].shape, activation='relu'), 
tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'), 
tf.keras.layers.MaxPool2D(pool_size=(2,2)), 
tf.keras.layers.Dropout(0.3), 
tf.keras.layers.Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'), 
tf.keras.layers.Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'), 
tf.keras.layers.MaxPool2D(pool_size=(2,2)), 
tf.keras.layers.Dropout(0.2), 
tf.keras.layers.Conv2D(128, kernel_size=(3,3), padding='same', activation='relu'), 
tf.keras.layers.Dropout(0.25), 
tf.keras.layers.Flatten(), 
tf.keras.layers.Dense(128, activation=tf.nn.relu), 
tf.keras.layers.BatchNormalization(), 
tf.keras.layers.Dropout(0.25), 
tf.keras.layers.Dense(10, activation=tf.nn.softmax), 
]) 
print("compiling")
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), 
loss=tf.keras.losses.sparse_categorical_crossentropy, 
metrics=['accuracy']) 
print("compilation end")

loading
loaded
preparing model


W1025 18:14:18.875933 21452 deprecation.py:506] From C:\Users\Admin\.conda\envs\neuro\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


compiling
compilation end


In [5]:
model.fit(x_train, y_train, epochs=3, verbose=1) 

Epoch 1/3
60000/60000 [==============================] - 56s 932us/sample - loss: 0.3046 - acc: 0.9068
Epoch 2/3
60000/60000 [==============================] - 48s 796us/sample - loss: 0.0801 - acc: 0.9755
Epoch 3/3
60000/60000 [==============================] - 48s 800us/sample - loss: 0.0583 - acc: 0.9824


In [5]:
model.save("compiled_mnist.h5")

In [5]:
model = tf.keras.models.load_model("./models/compiled_mnist.h5")

W1025 18:14:26.254213 21452 deprecation.py:506] From C:\Users\Admin\.conda\envs\neuro\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1025 18:14:26.257205 21452 deprecation.py:506] From C:\Users\Admin\.conda\envs\neuro\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1025 18:14:26.818847 21452 deprecation.py:506] From C:\Users\Admin\.conda\envs\neuro\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is de

In [10]:
for i in range(10):
    os.makedirs("./errors/" + str(i))

FileExistsError: [WinError 183] Невозможно создать файл, так как он уже существует: './errors/0'

In [9]:
def calculate_loss(ind, sess):
    w = model.predict(x_test[ind].reshape(1,28,28,1))
    loss_tensor = tf.keras.losses.sparse_categorical_crossentropy(y_test[ind], tf.nn.softmax(w))
    loss = loss_tensor.eval(session=sess)
    return loss[0]

In [10]:
def save_image(image, prediction, answer, index):
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111)
    description = "Image number %d in 'test' array\nRecognised as: %d;  Titled as:%d" % (index, prediction, answer)
    ax.set(title=description)
    ax.imshow(image, cmap="gray_r")
    path = "./errors/%d/%d.png" % (prediction, index)
    plt.savefig(path)
    
    

In [12]:
mistakes = []
sess = tf.compat.v1.Session()
for n, pic in enumerate(x_test[:5000]):
    prediction = model.predict(pic.reshape(1,28,28,1)).argmax()
    answer = y_test[n]
    if prediction != answer:
        print("\r%d of 60000" % n, end='')
        mistakes.append([n, calculate_loss(n, sess)])
        save_image(pic[:,:,0], prediction, answer, n)
sess.close()
print("Finished!")

4860 of 60000Finished!


In [18]:
#mistakes = sorted(mistakes, key=lambda x: x[1])
mistakes_indexes = np.array(mistakes).T[0].astype(int)

In [32]:
MISTAKE_WEIGHT = 3

old_loss, old_acc = model.evaluate(x_test, y_test)
model.fit(x_test[mistakes_indexes], 
          y_test[mistakes_indexes], 
          verbose=0,
          sample_weight=MISTAKE_WEIGHT*np.ones(mistakes_indexes.shape), 
          epochs=3)
new_loss, new_acc = model.evaluate(x_test, y_test)


10000/10000 [==============================] - 3s 273us/sample - loss: 0.1563 - acc: 0.9438


0.027507356682699175

In [ ]:
def show_image(image):
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111)
    ax.imshow(image.reshape(28,28), cmap="gray_r")